In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/learning-equality-curriculum-recommendations/sample_submission.csv
/kaggle/input/learning-equality-curriculum-recommendations/topics.csv
/kaggle/input/learning-equality-curriculum-recommendations/correlations.csv
/kaggle/input/learning-equality-curriculum-recommendations/content.csv


In [2]:
topics_df = pd.read_csv("/kaggle/input/learning-equality-curriculum-recommendations/topics.csv")
correlations_df = pd.read_csv("/kaggle/input/learning-equality-curriculum-recommendations/correlations.csv")
content_df = pd.read_csv("/kaggle/input/learning-equality-curriculum-recommendations/content.csv")

# topics_df
columns : 
+ id - unique_ids (string)
+ title - name of the topic (string)
+ description - description (55% data is null) (string)
+ channel - channel (string)
+ category - 'source', 'aligned', 'supplemental' (categorical)
+ level - 0, 1, 2, 3, 4, 5, 6, 7,  8,  9, 10 (0-10) (ordinal)
+ language - 'bg', 'en', 'pt', 'gu', 'my', 'zh', 'ar', 'te', 'es', 'fr', 'sw', 'mr', 'hi', 'bn', 'fil', 'ru', 'it', 'or', 'pnb', 'km', 'as', 'kn', 'ur', 'pl', 'ta', 'swa', 'tr', 'mul' (28 unique value) (categorical)
+ parent - 17.5k values (string)
+ has_content - true (79.9%) , false (20.1%) (Binary)


In [3]:
topics_df.head(2)

,id,title,description,channel,category,level,language,parent,has_content
0,t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True
1,t_000095e03056,Unit 3.3 Enlargements and Similarities,NaN,b3f329,aligned,2,en,t_aa32fb6252dc,False


# correlation_df
Relates the topics_df and content_df
+ topic_id - unique (string)
+ content_ids - list of contents related to the topic (list of strings)

In [4]:
correlations_df.head(2)

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
1,t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...


# content_df
Columns :
+ id -  unique_ids (string)
+ title -  name of the content (string)
+ description - description (42% data is null) (string)
+ kind - 'video', 'document', 'html5', 'exercise', 'audio' (categorical)
+ text - related text (52% null)
+ language - 'es', 'it', 'pt', 'en', 'mr', 'bg', 'gu', 'sw', 'hi', 'ar', 'bn','as', 'zh', 'fr', 'km', 'pl', 'ta', 'fil', 'or', 'ru', 'kn', 'swa', 'my', 'pnb', 'tr', 'te', 'ur' (categorical) (27 unique value)
+ copyright_holder (53% null)
+ license - nan, 'CC BY-NC-ND','CC BY-NC-SA', 'Public Domain', 'CC BY', 'CC BY-SA', 'CC BY-NC', 'CC BY-ND'(52% null)

In [5]:
content_df.head(2)

,id,title,description,kind,text,language,copyright_holder,license
0,c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,NaN,es,NaN,NaN
1,c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,NaN,it,NaN,NaN


In [6]:
# let's clean the content_df first 
content_df = content_df.drop(columns=['description' , 'text' , 'copyright_holder' , 'license' , 'title'])
topics_df = topics_df.drop(columns=['title' , 'description'])

## Preoprocessing
content_df 
+ OneHotEncoder - Kind , language

topics_df
+ OneHotEncoder - category , language
+ LabelEncoder - has_content
+ OrdinalEncoder - level